In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf


# Start spark session
spark = SparkSession.builder.appName("ExoSearch").getOrCreate()

In [2]:
exotrain_df = spark.read.csv(path='exoTrain_new.csv', header=True, inferSchema=True)

In [3]:
exotrain_df.limit(1000).toPandas().head()

,LABEL,FLUX1,FLUX2,FLUX3,FLUX4,FLUX5,FLUX6,FLUX7,FLUX8,FLUX9,...,FLUX3188,FLUX3189,FLUX3190,FLUX3191,FLUX3192,FLUX3193,FLUX3194,FLUX3195,FLUX3196,FLUX3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [4]:
# 1 has confirmed to be only one planet system and 2 means that it can be multi-planet system

# How many stars there is in a dataset
exotrain_df.count()

5087

In [12]:
empty_dict = [exotrain_df.filter(exotrain_df[str(col)].isNull()).count() for col in exotrain_df.columns]

In [16]:
# Check how many columns have null values
print([x for x in empty_dict if x!=0])

[]


## Begin machine learning part with spark

In [8]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [11]:
old_columns = exotrain_df.columns

assembler = VectorAssembler(inputCols=old_columns, outputCol="features")
assembled_df = assembler.transform(exotrain_df)

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# fit and transform
scaled_model = scaler.fit(assembled_df)
scaled_df = scaled_model.transform(assembled_df)

In [ ]:
#empty_dict = {exotrain_df.filter(exotrain_df[str(col)].isNull()).count() for col in exotrain_df.columns}